# Segmentation
Segmentate a file raster image following the SLIC algorithm

In [23]:
import numpy as np
import gdal
from skimage.segmentation import slic
from skimage import exposure #print(skimage.__version__)
import time

print('Libraries imported successfully')

Libraries imported successfully


In [ ]:
seg_start=time.time() #For checking ejecution time

**Define directories**

 - <b>tmp_raster</b>: name of the file for the segmentation proccess
 
 - <b>raser_path</b>: path of the raster image
  
 - <b>output</b>: output path to save the image

In [ ]:
tmp= '/home/jovyan/work/DATASET/'

tmp_raster = 'valenciana_704000.0-4400000.0_comunidad_mask.tif'
raster_path = f'{tmp}Dataset_processing/Tiling/{tmp_raster}'
output = f'{tmp}OBIA/Segmentation/segmented_{tmp_raster}.tif'

**Read** raster image

In [ ]:
driverTiff = gdal.GetDriverByName('GTiff')
raster_ds = gdal.Open(raster_path)

**Append** all band together

In [ ]:
nbands = raster_ds.RasterCount
band_data = []
for i in range(1, nbands+1):
    band = raster_ds.GetRasterBand(i).ReadAsArray()
    band_data.append(band)

In [ ]:
band_data = np.dstack(band_data)
img = exposure.rescale_intensity(band_data)
img = img.astype('double')

Define variables and process the **SLIC algorithm**

In [ ]:
segments = slic(img, n_segments=10000, compactness=0.1, sigma=1, start_label=1)
print(f"SLIC number of segments: {len(np.unique(segments))}")

**Save segments** to raster

In [ ]:
segments_ds = driverTiff.Create(output, raster_ds.RasterXSize, raster_ds.RasterYSize,
                                1, gdal.GDT_Float32)
segments_ds.SetGeoTransform(raster_ds.GetGeoTransform())
segments_ds.SetProjection(raster_ds.GetProjectionRef())
segments_ds.GetRasterBand(1).WriteArray(segments)
segments_ds = None

In [ ]:
print(f'Segment image finished successfully in {time.time() - seg_start} s')